In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import cross_validate
import re
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder
import re
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [2]:
data = pd.read_csv('processed_df.csv')

**ВЫКИНЕМ НАНЫ**

In [3]:
data_no_nans = data.copy()
data_no_nans = data_no_nans.dropna()
data_no_nans.isna().sum()

Unnamed: 0              0
Maruti                  0
Skoda                   0
Honda                   0
Hyundai                 0
Toyota                  0
Ford                    0
Renault                 0
Mahindra                0
Tata                    0
Chevrolet               0
Fiat                    0
Datsun                  0
Jeep                    0
Mercedes-Benz           0
Mitsubishi              0
Audi                    0
Volkswagen              0
BMW                     0
Nissan                  0
Lexus                   0
Jaguar                  0
Land                    0
MG                      0
Volvo                   0
Daewoo                  0
Kia                     0
Force                   0
Ambassador              0
Ashok                   0
Isuzu                   0
Opel                    0
year                    0
selling_price           0
km_driven               0
Diesel                  0
Individual              0
Dealer                  0
Manual      

In [4]:
scaller_x = StandardScaler()
x = scaller.fit_transform(data_no_nans.loc[:, data_no_nans.columns != 'selling_price'])

NameError: name 'scaller' is not defined

In [ ]:
scaller.

In [ ]:
scaller_x = StandardScaler()
x = scaller.fit_transform(data_no_nans.loc[:, data_no_nans.columns != 'selling_price'])

y = scaller.fit_transform(data_no_nans['selling_price'].to_numpy())

In [ ]:
data_scld = pd.DataFrame(data=np.hstack((y, x)), columns=['selling_price'] + list())

**ВЫДЕЛИМ ТРЕНИРОВОЧНЫЕ И ТЕСТОВЫЕ МАССИВЫ**

In [ ]:
train_data_no_nans = data_no_nans.sample(frac=0.8)
test_data_no_nans = data_no_nans.iloc[~data_no_nans.index.isin(train_data_no_nans.index), :]

In [ ]:
features_train = train_data_no_nans.iloc[:, train_data_no_nans.columns != 'selling_price']
responses_train = train_data_no_nans['selling_price'].to_frame(name='selling_price')
features_test = test_data_no_nans.iloc[:, test_data_no_nans.columns != 'selling_price']
responses_test = test_data_no_nans['selling_price'].to_frame(name='selling_price')

**НОРМИРУЕМ ДАННЫЕ**

In [ ]:
scaller = StandardScaler()
x_train = scaller.fit_transform(features_train)
y_train = responses_train
x_test = scaller.fit_transform(features_test)
y_test = responses_test

# **ОБУЧИМ МОДЕЛЬ НА ТЕХ ДАННЫХ, ЧТО ЕСТЬ**

**ОБУЧИМ МОДЕЛЬ С ЛИНЕАРИЗАЦИЕЙ КОЭФФИЦИЕНТОВ L1 LASSO**

In [ ]:
l1_regr = Lasso(1e-5, max_iter=10 ** 4)
l1_regr.fit(x_train, y_train)
l1_regr.score(x_train, y_train)

In [ ]:
l1_regr.coef_

In [ ]:
l1_regr.score(x_test, y_test)

**КАК ВИДНО НА ТЕСТОВОЙ ВЫБОРКЕ РЕЗУЛЬТАТ ДАЖЕ ЛУЧШЕ.**

**МФ ПОЛУЧИЛИ РЕЗУЛЬТАТ ПО МЕТРИКЕ R2**

**ПОПРОБУЕМ ОЦЕНИТЬ С ПОМОЩЬЮ МЕТРИКИ MAE**

In [ ]:
y_train_predict = l1_regr.predict(x_train)
y_test_predict = l1_regr.predict(x_test)
loss_train = mean_absolute_error(y_train, y_train_predict)
loss_test = mean_absolute_error(y_test, y_test_predict)

In [ ]:
print(loss_train)
print(loss_test)

**ПОСМОТРИМ СКОЛЬКО ЭТО В ДЕНЕЖНОМ ЭКВИВАЛЕНТЕ С ПОМОЩЬЮ ИНВЕРСИИ НОРМАЛИЗАЦИИ**

**ОЦЕНИМ С ПОМОЩЬЮ МЕТРИКИ MAPE**

In [ ]:
loss_train_mape = mean_absolute_percentage_error(y_train, y_train_predict)
loss_test_mape = mean_absolute_percentage_error(y_test, y_test_predict)

In [ ]:
print(loss_train_mape)
print(loss_test_mape)

**ОПЯТЬ ЗАМЕЧАЕМ, ЧТО НА ТЕСТОВЫХ ДАННЫХ РЕЗУЛЬТАТ ЛУЧШЕ**

# **ДОБАВИМ ПОЛИНОМ 9 СТЕПЕНИ ПО ПРИЗНАКАМ ТИП ТОПЛИВА И ОБЬЕМ ДВИГАТЕЛЯ**

**В РЕЗУЛЬТАТЕ АНАЛИЗА ДАННЫХ (ФАЙЛ processing_nes.ipynb) МЫ ВЫЯСНИЛИ, ЧТО ОБЬЕМ ДВИГАТЕЛЯ НАХОДИТСЯ В НЕКОТОРОЙ КОРЕЛЯЦИИ ОТ ТИПА ТОПЛИВА. ПОСТРОИВ ПОЛИНОМИАЛЬНУЮ МОДЕЛЬ, ОБУЧИВ ЕЕ И ПОСЧИТАВ ПОТЕРИ С КРОСС ВАЛИДАЦИЕЙ МЫ ПОНЯЛИ, ЧТО ДЛЯ ДВУХ ЭТИХ ДВУХ ПРИЗНАКОВ С ПОЛИНОМОМ СТЕПЕНЬЮ 9 (ФАЙЛ analyse.ipynb) МЫ МОЖЕМ УЛУЧШИТЬ МОДЕЛЬ. ПОПРОБУЕМ ЕЕ УЛУЧШИТЬ ДОБАВИВ ЭТИ ПРИЗНАКИ**

**ВОЗЬМЕМ ФРЕЙМ БЕЗ НАНОВ И СКАЛЯРИЗУЕМ**

In [ ]:
scaller_poly = StandardScaler()
x_poly = scaller_poly.fit_transform(data_no_nans[data_no_nans.columns.drop('selling_price')])
y_poly = scaller_poly.fit_transform(data_no_nans[['selling_price']])

In [ ]:
scaled_df = pd.DataFrame(data=np.hstack((y_poly, x_poly)), columns=['selling_price'] + list(data_no_nans.columns.drop('selling_price')))
scaled_df

**СОЗДАДИМ ПОЛИНОМ 9Й СТЕПЕНИ ДЛЯ ДВИГАТЕЛЯ И ДИЗЕЛЯ**

In [ ]:
poly_obj = PolynomialFeatures(7)
engine_diesel_poly = poly_obj.fit_transform(scaled_df[['Diesel', 'engine']])
engine_diesel_poly.shape

**УБЕРЕМ ПЕРВЫЕ ТРИ СТОЛБЦА, ПЕРВЫЙ ПОТОМУ ЧТО ЕДИНИЧНЫЙ ДЛЯ СВОБОНОГО ВЕСА, ВТОРЫЕ ДВА ПОТОМУ ЧТО ПОВТОРЕНИЕ ТЕХ, ЧТО УЖЕ ЕСТЬ В ДАТАСЕТЕ, ЗАПИШЕМ ВО ФРЕЙМ**

In [ ]:
engine_diesel_poly_df = pd.DataFrame(data=engine_diesel_poly[:, 3:], columns=[f'Dies*Engine{i + 1}' for i in range(engine_diesel_poly.shape[1] - 3)])

**СОБЕРЕМ ПОЛИНОМ С ДАТАСЕТОМ**

In [ ]:
poly_prepared = pd.concat([scaled_df, engine_diesel_poly_df], axis=1)

**РАЗДЕЛИМ ВЕСЬ СКАЛЯРИЗОВАННЫЙ ДАТАСЕТ НА ТРЕНИРОВОЧНЫЙ И ТЕСТОВЫЙ**

In [ ]:
poly_prepared_train = poly_prepared.sample(frac=0.8, random_state=0).sort_index()
poly_prepared_test = poly_prepared.iloc[~scaled_df.index.isin(poly_prepared_train.index)].sort_index()

**НАЧНЕМ ОБУЧАТЬ МОДЕЛЬ С L1/L2 НА ТРЕНИРОВОЧНОМ ДАТАСЕТЕ**

In [ ]:
regr_poly = Ridge(alpha=0.0001, max_iter=10 ** 4)
regr_poly.fit(poly_prepared_train.iloc[:, 1:], poly_prepared_train.iloc[:, 0])

In [ ]:
regr_poly.coef_

**ОЦЕНИМ ПОТЕРИ ПО МЕТРИКЕ MAE**

In [ ]:
y_train_predict = regr_poly.predict(poly_prepared_train.iloc[:, 1:])
y_test_predict = regr_poly.predict(poly_prepared_test.iloc[:, 1:])
loss_train = mean_absolute_error(poly_prepared_train.iloc[:, 0], y_train_predict)
loss_test = mean_absolute_error(poly_prepared_test.iloc[:, 0], y_test_predict)

In [ ]:
print(loss_train)
print(loss_test)

In [ ]:
print(scaller_poly.inverse_transform([[loss_train]]))
print(scaller_poly.inverse_transform([[loss_test]]))

**ОЦЕНИМ ПОТЕРИ ПО МЕТРИКЕ MAPE**

In [ ]:
loss_train_mape = mean_absolute_percentage_error(poly_prepared_train.iloc[:, 0], y_train_predict)
loss_test_mape = mean_absolute_percentage_error(poly_prepared_test.iloc[:, 0], y_test_predict)
print(loss_train_mape)
print(loss_test_mape)

**КАК ВИДНО РЕЗУЛЬТАТЫ ЧУУУУУУУТЬ ЧУТЬ УЛУЧШИЛИСЬ, ПОПРОБУЕМ ЕЩЕ УЛУЧШИТЬ МОДЕЛЬ**

# **МОЩНОСТЬ, ТИП ДВИГАТЕЛЯ И КРУТЯЩИЙ МОМЕНТ**

In [ ]:
data = pd.read_csv('processed_df.csv')
data = data.dropna()

**ВИДЕЛИМ ПРИЗНАКИ И ОТКЛИКИ**

In [ ]:
x = data.loc[:, data.columns != 'selling_price']
y = data[['selling_price']]

**СКАЛЯРИЗУЕМ ПРИЗНАКИ И ОТКЛИКИ**

In [ ]:
scall = StandardScaler()
x_scld = scall.fit_transform(x)
y_scld = scall.fit_transform(y)

In [ ]:
features_df = pd.DataFrame(data=x_scld, columns=x.columns)

**СОЗДАДИМ ПОЛИНОМ 3Й СТЕПЕНИ ПО ТИПУ ТОПЛИВА, МОЩНОСТИ И КРУТЯЩЕМУ МОМЕНТУ**

In [ ]:
poly3 = PolynomialFeatures(3)
x_poly = poly3.fit_transform(features_df.loc[:, ['Diesel', 'max_power', 'torque']])[:, 4:]

**СОЗДАДИМ ОБЩИЙ ФРЕЙМ ПРИЗНАКОВ**

In [ ]:
poly_df = pd.DataFrame(data=x_poly, columns=[f'Dies*Pow*Tor{i + 1}' for i in range(x_poly.shape[1])])
resp_df = pd.DataFrame(data=y_scld, columns=['selling_price'])

In [ ]:
poly_df = pd.concat([resp_df, features_df, poly_df], axis=1)

**ВЫДЕЛИМ ТРЕНИРОВОЧНЫЙ И ТЕСТОВЫЙ ДАТАСЕТЫ**

In [ ]:
train = poly_df.sample(frac=0.8, random_state=0)
test = poly_df.iloc[~poly_df.index.isin(train.index), :]

**ПОПРОБУЕМ ОБУЧИТЬ МОДЕЛЬ С L1 И ПОСМОТРИМ НА РЕЗУЛЬТАТ РАЗНЫМИ МЕТРИКАМИ**

In [ ]:
lin_regr = Ridge(alpha=0.000001, max_iter=10000)
lin_regr.fit(train.loc[:, train.columns != 'selling_price'], train['selling_price'])

In [ ]:
lin_regr.coef_

In [ ]:
y_predict_train = lin_regr.predict(train.loc[:, train.columns != 'selling_price'])
y_predict_test = lin_regr.predict(test.loc[:, test.columns != 'selling_price'])
mae_train = mean_absolute_error(train['selling_price'], y_predict_train)
mae_test = mean_absolute_error(test['selling_price'], y_predict_test)

In [ ]:
print(mae_train)
print(mae_test)

In [ ]:
print(scall.inverse_transform([[mae_train]]))
print(scall.inverse_transform([[mae_test]]))